#Import der notwenigen Pakete

In [40]:
from pylab import *
import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button, RadioButtons

In [13]:
def prepGoL (size = [9,9], bouncon = 1, incon = [[3,3],[3,4],[3,5]], rule = 'B3/S23'):
    """Die Funktion prepGoL bereitet Textdateien vor, in welche die Bedingungen zum Ausführen von GameOfLife benötigt werden.
    Es sind standrtmäßige Werte eingesetzt, welche nach belieben verändert werden können. Die Veränderung können sowohl durch Veränderung der Parameter der Funktion oder durch Bearbeitung des erstellten Dokuments vorgenommen werden.  
    kwargs:
        size = [9,9] : Systemgröße
        bouncon = 1 : Randbedingungen
        incon = [[3,3],[3,4],[3,5]] : Anfangsbedingungen
        rule = 'B3/S23' : Regelstring
        
    Hinweise: Es ist eine Form nach folgendem Muster für die Anweisungen zu wählen:
        size: [Länge, Breite]
        bouncon:
            0 - periodisch
            1 - begrenzt
            2 - unendlich (dynamisch)
        incon: [[xpos,ypos], [xpos,ypos], ...]
        rule: B3/S23
            B - Anzahlen von Nachbarzellen zur Geburt - /S - Anzahlen von Nachbarzellen zum Überleben vorhandener Zellen
    kein return Wert
    """
    a = open('GameOfLifeSize.dat','w')
    a.write('#Systemgröße als zwei natürliche Zahlen; Höhe und Breite (Höhe Breite)\n')
    for i in range(2):
        a.write('{} '.format(size[i]))
    a.write("\n")
    b = open('GameOfLifeBouncon.dat','w')
    b.write('#Randbedingungen als Zahl:\n')
    b.write('#0 - periodisch\n')
    b.write('#1 - begrenzt\n')
    b.write('#2 - unendlich\n')
    b.write('{}\n'.format(bouncon))
    c = open('GameOfLifeIncon.dat','w')
    c.write('#Anfangsbedingungen; Positionen der einzelnen besetzten Zellen untereinander: xpos ypos\n')
    for i in range(len(incon)):
        for j in range(2):
            c.write('{} '.format(incon[i][j]))
        c.write("\n")
    d = open('GameOfLifeRule.dat','w')
    d.write('#Regeln für das Leben der Zellen: z.B. "3 ENTER 2 3"\n')
    d.write('#erste Zeile: Anzahlen von Nachbarzellen zur Geburt \n#zweite Zeile: Anzahlen von Nachbarzellen zum Überleben vorhandener Zellen\n')
    d.write('#Anschließend sind die Zeilen so mit 0en aufzufüllen, dass sie gleich lang sind')
    for i in range(3):
        for j in range(max(len(rule.split("/")[1])-1, len(rule.split("/")[0])-1)):
            try:
                d.write('{} '.format(int(rule.split("/")[i-1][j+1])))
            except:
                d.write('0')
        d.write("\n")
    a.close()
    b.close()
    c.close()
    d.close()

In [14]:
prepGoL(size = [9,9], bouncon = 2, incon = [[3,3],[3,4],[3,5]], rule = 'B3/S23')

##Laufzeittest, ob eine Summe oder eine Matrixoperation schneller ist.

In [5]:
import numpy as np
k=10110
M=np.ones([k,k]).astype(int)
%time print(sum(M[:,0]))
print(M[:,0])
%time print(M[:,0].dot(np.ones(k)))

10110
Wall time: 10.5 ms
[1 1 1 ..., 1 1 1]
10110.0
Wall time: 1 ms


##Es wird also weiter mit Matrixoperationen gearbeitet

In [15]:
def GoLStep(M, livecrit, borncrit, bouncon):
    """
    Die Funktion GoLStep führt einen einzelnen Schritt des Programms GameOfLife durch.
    args:
        M : Matrix, welche die Daten des Feldes entählt (0 für tote und 1 für lebende Zellen)
        livecrit : Liste mit Kriterien, wann eine lebende Zelle am Leben bleibt. (Liste mit int Zahlen)
        borncrit : Liste mit Kriterien, wann eine neue Zelle geboren wird. (Liste mit int Zahlen)
        bouncon : Randbedingungen (int Zahl)
                    0 - periodisch
                    1 - begrenzt
                    2 - unendlich (dynamisch) 
    
    returns newM (neu, nach den vorgegebenen Regeln berechnete Matrix / neues Feld)
    """
    hohe, breite=M.shape
    if bouncon==2:
        l = M[:,0].dot(np.ones(hohe)) #links
        r = M[:,-1].dot(np.ones(hohe)) #rechts
        o = M[0].dot(np.ones(breite)) #oben
        u = M[-1].dot(np.ones(breite)) #unten
        if not l==0:
            M = np.c_[np.zeros(hohe), M]
            breite=breite+1
        if not r==0:
            M = np.c_[M, np.zeros(hohe)]
            breite=breite+1
        if not o==0:
            M = np.r_[np.zeros([1,breite]),M]
            hohe=hohe+1
        if not u==0:
            M = np.r_[M,np.zeros([1,breite])]
            hohe=hohe+1

        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
    elif bouncon==1:
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
    elif bouncon==0:
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor1[0,-1]=1
        neighbor1[-1,0]=1
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
        neighbor2[0,-1]=1
        neighbor2[-1,0]=1
    neighbors = neighbor1.dot(M).dot(neighbor2)-M
    livenei = M*neighbors #Matrix, welche die Nachbarzahlen der lebenden Zellen enthält
    deadnei = neighbors*(1-M) #Matrix, welche die Nachbarzahlen der toten Zellen enthält
    newM = np.zeros((hohe,breite))
    for i in livecrit:
        newM[livenei==i]=1
    for i in borncrit:
        newM[deadnei==i]=1
    #print(M)
    #print(breite, hohe)
    #print(newM)
    return newM, breite, hohe

In [175]:
def GoLplot(field, step, livecrit, borncrit, bouncon, interval, speed):
    """
    Die Funktion GoLplot stellt die Entwicklung des Feldes nach vorgegebener Regel dar.
    args:
        field : Ausganagsmatrix (Feld zum Schritt 0) mit den Daten der lebenden Zellen (1) und der toten Zellen (0)
        step : Funktion, welche die Matrix zum nächsten Zeitpunkt bestimmt (mit den args *[field, livecrit, borncrit, bouncon])
        livecrit : Liste mit Kriterien, wann eine lebende Zelle am Leben bleibt. (Liste mit int Zahlen)
        borncrit : Liste mit Kriterien, wann eine neue Zelle geboren wird. (Liste mit int Zahlen)
        bouncon : Randbedingungen (int Zahl)
                    0 - periodisch
                    1 - begrenzt
                    2 - unendlich (dynamisch)  
        interval : Anzahl der durchzuführenden Schritte (int Zahl)
        speed : Schnelligkeit, mit der die Animation gezeigt werden soll
                    float Zahl, welche die Verweildauer in ms angibt 
    
    kein return Wert
    """
    fig, ax = subplots()
    ax.set_position([0.3,0.25,0.69,0.69])
    line = ax.imshow(field, interpolation='nearest')
    sfield, field2 = np.array(field), np.array(field)
    fields=[np.array(field2)]
    ihohe, ibreite=sfield.shape
    hohen=[ihohe]
    breiten=[ibreite]
    def animate(i):
        if len(fields)>=i+1:
            newfield=fields[i]
            hohe=hohen[i]
            breite=breiten[i]
        else:            
            field=fields[i-1]         
            newfield, breite, hohe = step(field, livecrit, borncrit, bouncon)
            fields.append(np.array(newfield))
            hohen.append(int(hohe))
            breiten.append(int(breite))
        #print()
        #print(i)
        #print(len(fields))
        ax.set_xlim([-0.5, breite-0.5])
        ax.set_ylim([-0.5, hohe-0.5])
        ax.clear()
        line = ax.imshow(newfield, interpolation='nearest')
        ax.set_title("$step$ = {}".format(i))
        return line

    #Optionsfunktionen
    def Option(option):
        slider1.set_visible(0)
        slider2.set_visible(0)
        slider3.set_visible(0)
        box.set_visible(0)
        if option=="general Options":
            pass #Randbedingungen, Regelstring, Size, wenn Randbedingungen Endlich...
        if option=="Colors":
            slider1.set_visible(1)
            slider2.set_visible(1)
            slider3.set_visible(1)
            box.set_visible(1)
            r.on_changed(colorslide)
            g.on_changed(colorslide)
            b.on_changed(colorslide)
        if option=="Running":
            pass
        if option=="Speed":
            pass
        if option=="Edit field":
            pass
        if option=="Reset All":
            pass
    
    #Farben
    red=0
    green=0
    blue=0
    def colorslide(val):
        red=r.val
        green=g.val
        blue=b.val
        col = {'red': [(0.0,  0.0, 0.0),
                       (1.0,  red, red)],

             'green': [(0.0,  0.0, 0.0),
                       (1.0,  green, green)],

             'blue':  [(0.0,  0.0, 0.0),
                       (1.0,  blue, blue)],

             'alpha': [(0.0,  0.0, 0.0),
                       (1.0,  1.0, 1.0)] }
        cm = mpl.colors.LinearSegmentedColormap("cm", col)
        register_cmap(cmap=cm)
        set_cmap(cm)
        
        
    def colorbut(col):
        if col=="red":
            r.val=1
            g.val=0
            b.val=0
        if col=="green":
            g.val=1
            r.val=0
            b.val=0
        if col=="blue":
            b.val=1
            r.val=0
            g.val=0
        if col=="black":
            r.val=0
            g.val=0
            b.val=0
        if col=="yellow":
            r.val=1
            g.val=1
            b.val=0
        if col=="Personal":
            pass
        colorslide(0)
    
    #Optionen anzeigen
    opt = axes([0.025, 0.65, 0.25, 0.25])
    Options = RadioButtons(opt, ('General Options', 'Manual Running', 'Edit field', 'Colors'), active=0)
    Options.on_clicked(Option)
    resetsa = plt.axes([0.025, 0.6, 0.25, 0.04])
    resetsb = Button(resetsa, 'Reset All')
    
    #Optionselemente
    slider1 = axes([0.35, 0.1, 0.6, 0.03])
    slider1.tick_params(labelleft='off',labelbottom='off', bottom='off', top='off', left='off', right='off')
    slider2 = axes([0.35, 0.15, 0.6, 0.03])
    slider2.tick_params(labelleft='off',labelbottom='off', bottom='off', top='off', left='off', right='off')
    slider3 = axes([0.35, 0.05, 0.6, 0.03])
    slider3.tick_params(labelleft='off',labelbottom='off', bottom='off', top='off', left='off', right='off')
    box = axes([0.025,0.2,0.25,0.38])
    reseta = axes([0.025,0.025,0.2,0.1])
    resetb = Button(reseta, 'Reset')
    
    #Colors
    r = Slider(slider1, 'red', 0.0, 1.0, valinit=red)
    g = Slider(slider2, 'green', 0.0, 1.0, valinit=green)
    b = Slider(slider3, 'blue', 0.0, 1.0, valinit=blue)
    
    #Anfangsbedingung schaffen
    Option("General Options")
    colorbut("black")
    ani = animation.FuncAnimation(fig, animate, np.arange(0, interval+1),
                                  interval=speed, repeat=True, repeat_delay=speed*5)
    show()

In [177]:
GameOfLife(steps=100, speed=0)

##Funktion zum Import der Benutzereingabe

In [17]:
def GoLdataimport():
    """
    Die Funktion GoLdataimport importiert die Bedingungen aus den mit prepGoL erstellten Textdateien. Die Form ist wie bei prepGoL beschrieben zu halten.
    Sie benötigt keine Argumente.
    returns (np.arrays)
        size : Größe des Feldes
        bouncon : Randbedingung
        incon : Anfangsbedingung
        rule : Regel (erste Zeile: bornrule, zweite Zeile: liverule)
    """
    size = np.loadtxt("GameOfLifeSize.dat", dtype=int)
    bouncon = np.loadtxt("GameOfLifeBouncon.dat", dtype=int)
    incon = np.loadtxt("GameOfLifeIncon.dat", dtype=int)
    rule = np.loadtxt("GameOfLifeRule.dat", dtype=int)
    return size, bouncon, incon, rule

In [7]:
size, bouncon, incon, rule=GoLdataimport()

##Funktion, welche den Datenimport und den Plot vornimmt

In [18]:
def GameOfLife(steps=10, speed=1):
    """
    Die Funktion GameOfLife erzeugt eine graphisch animierte Ausgabe der Entwicklung der Anfangsmatrix nach dem Game of Life.
    Die Bedingungen sind durch Textdateien vorzugeben (siehe auch prepGoL).
    kwargs:
        steps=10 : Anzahl der durchzufürenden Schritte
        speed=1 : Geschwindigkeit, mit der die Aimation erfolgen soll
                    100*10**speed ist die Zeit in ms pro Schritt (empfohlen: Wert zwischen 0 (langsam) und 1 (schnell))
    """
    size, bouncon, incon, rule = GoLdataimport()
    liverule = [rule[1][i] for i in range(len(rule[1])) if rule[1][i]!=0]
    bornrule = [rule[0][i] for i in range(len(rule[0])) if rule[0][i]!=0]
        
    
    field =  np.zeros(size)
    for i in range(int(incon.size/incon[0].size)):
        field[incon[i][1], incon[i][0]]=1
    
    speed=100*10**(1-speed)
    
    GoLplot(field, GoLStep, liverule, bornrule, bouncon, steps, speed)
    return

In [12]:
GameOfLife(steps=100, speed=0)

Problem: Er rechnet immer vom Ende weiter, die Anfangsbedungung ist also noch zu speichern. - gelöst

Ferner funktionrt die Unendlichkeitsbedingung noch nicht.
    - Erweiterung der Matrix wird nicht angezeigt und nicht übernommen - gelöst

In [98]:
np.c_[np.ones([2,3]), np.zeros([2,3])]

array([[ 1.,  1.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.]])

In [13]:
x=np.ones([3,3])
print(x)
y=np.array(x)

y.resize([33,33])
x.data=y
print(y)
print(x)
print(np.array(x.data))

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]


In [144]:
a=np.ones([2,2])
a.data=np.zeros([3,3])
a[1,1]=2
print(a)
a.base

[[ 0.  0.]
 [ 0.  2.]]


array([[ 0.,  0.,  0.],
       [ 2.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [53]:
x=[1,2,3,4,5]
x.append(6)
x[5]

6

Das Grundprinzip funktioniert nun. Es fehlen einige Aspekte wie
- Randbedingungen in GoLStep
- Funktionsbeschreibungen
- evtl. Sonderfunktionen wie 
    - Farbwahl
    - Starten, Stoppen per Maus
    - Bearbeiten der Matrix per Maus
    - ...